# Quantum Relative Entropy

Here's an explanation of how the function works:

1. We import the necessary libraries: Cirq for the quantum circuit simulation and NumPy for numerical operations.

2. We define the function quantum_relative_entropy that takes two Cirq quantum circuits as input.

3. obtain their corresponding density matrices rho1 and rho2.

4. We calculate the quantum relative entropy using the formula:

$$
S(\rho_1 | \rho_2) = \mathbf{Tr}(\rho_1(\log_2(\rho_1+\delta)- \log_2(\rho_2+\delta)))
$$

where delta is a small positive constant that ensures the logarithms are well-defined for zero eigenvalues.

5. We return the quantum relative entropy as the output of the function.

Note that the function assumes that the two input circuits have the same number of qubits, since the density matrices must have the same dimensionality to be compared using the quantum relative entropy. If the circuits have different numbers of qubits, you may need to modify the function to handle this case appropriately.

In [8]:
import cirq
import numpy as np

def quantum_relative_entropy(circuit1, circuit2):
    # Create a simulator to simulate the circuits
    simulator = cirq.Simulator()

    # Simulate the circuits to obtain the corresponding density matrices
    rho1 = cirq.final_density_matrix(circuit1)
    rho2 = cirq.final_density_matrix(circuit2)

    # Calculate the quantum relative entropy
    d = rho1.shape[0]
    delta = np.eye(d) / d
    rel_entropy = np.trace(rho1 @ (np.log2(rho1 + delta) - np.log2(rho2 + delta)))

    return rel_entropy


The code defines a function graph_state_circuit that takes a positive integer num_qubits as input and returns a Cirq quantum circuit on num_qubits qubits. The circuit is initialized with Hadamard gates applied to all qubits, and then a graph with num_qubits nodes is randomly generated. Random controlled unitary gates are then applied to each edge of the graph, where the unitary matrix is generated using the cirq.testing.random_unitary function. The function returns the resulting circuit. This generalizes graph states and can be thought of as a perturbation of a graph state. We would like to compare two such perturbed graph states by computing their quantum relative entropies. Note, they must have the same number of qubits. 

In [16]:
import cirq
import numpy as np

def graph_state_circuit(num_qubits):
    """Returns a graph state circuit on num_qubits qubits"""
    # Generate a random graph with num_qubits nodes
    graph = nx.fast_gnp_random_graph(num_qubits, 0.5)

    # Initialize the circuit with the given qubits
    qubits = [cirq.LineQubit(i) for i in range(num_qubits)]
    circuit = cirq.Circuit()

    # Apply Hadamard gates to all qubits
    circuit.append(cirq.H.on_each(qubits))

    # Apply random controlled unitary gates to each edge
    for edge in graph.edges():
        control_qubit, target_qubit = qubits[edge[0]], qubits[edge[1]]

        # Generate a random unitary matrix
        random_unitary = cirq.testing.random_unitary(2)

        # Apply the controlled random unitary gate to the circuit
        circuit.append(cirq.ControlledGate(cirq.MatrixGate(random_unitary)).on(control_qubit, target_qubit))

        
    # Define number of qubits in circuit
    circuit.num_qubits = len(circuit.all_qubits())

    
    return circuit


In [17]:
num_qubits = 4
circuit1 = graph_state_circuit(num_qubits)
circuit1

0: ───H───@───────────────────────────────────────────────────────────────────────────────────────────────
          │
          ┌                           ┐
1: ───H───│ 0.409-0.656j  0.632+0.051j│───@───────────────────────────────────────────────────────────────
          │ 0.205-0.6j   -0.765+0.113j│   │
          └                           ┘   │
                                          │
                                          ┌                           ┐
2: ───H───────────────────────────────────│-0.755+0.445j -0.177+0.447j│───@───────────────────────────────
                                          │ 0.262+0.404j -0.828-0.288j│   │
                                          └                           ┘   │
                                                                          │
                                                                          ┌                           ┐
3: ───H───────────────────────────────────────────────────────────────────│ 0.544+0.506j  0.498-0.446j│───
                                                                          │-0.311-0.592j  0.687-0.284j│
                                                                          └                           ┘

In [18]:
circuit2 = graph_state_circuit(num_qubits)
circuit2

0: ───H───@───────────────────────────────@───────────────────────────────────────────────────────────────────────────────────────────────
          │                               │
          ┌                           ┐   │
1: ───H───│-0.191-0.695j -0.666+0.191j│───┼───────────────────────────────@───────────────────────────────────────────────────────────────
          │ 0.084-0.688j  0.717+0.079j│   │                               │
          └                           ┘   │                               │
                                          │                               │
2: ───H───────────────────────────────────┼───────────────────────────────┼───────────────────────────────@───────────────────────────────
                                          │                               │                               │
                                          ┌                           ┐   ┌                           ┐   ┌                           ┐
3: ───H───────────────────────────────────│-0.245-0.259j  0.185+0.916j│───│ 0.144+0.025j  0.156+0.977j│───│-0.615+0.603j  0.508+0.021j│───
                                          │-0.737-0.575j -0.122-0.335j│   │ 0.073-0.987j -0.141+0.037j│   │-0.349+0.369j -0.861-0.002j│
                                          └                           ┘   └                           ┘   └                           ┘

In [19]:
import cirq

# Test the quantum_relative_entropy function
rel_entropy = quantum_relative_entropy(circuit1, circuit2)
print(f"Quantum relative entropy: {rel_entropy}")

Quantum relative entropy: (18.366376192416+4.440892098500626e-16j)
